In [67]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, max
from pyspark.sql.functions import avg
import json
import csv

In [16]:
# 1.Task (RDD):

# Read a text file containing paragraphs and find the paragraph with the maximum number of words.
# Hint: Use the textFile transformation, split paragraphs into words, and find the paragraph with the most words using the max action.

In [13]:
# Initialize SparkSession
spark = SparkSession.builder.appName("WordCount").getOrCreate()
sc = spark.sparkContext

In [14]:
# Read the text file as an RDD
text_rdd = sc.textFile("C:\\Ieva\\Python_Bootcamp\\Task folder\\29.08.2023_tasks\\Random_text.txt")

In [17]:
# Find the paragraph with the most words using the max action
max_words_paragraph = text_rdd \
    .map(lambda paragraph: (paragraph, len(paragraph.split()))) \
    .max(key=lambda x: x[1])

In [18]:
print("Paragraph with the most words:")
print(max_words_paragraph[0])

Paragraph with the most words:
Far concluded not his something extremity. Want four we face an he gate. On he of played he ladies answer little though nature. Blessing oh do pleasure as so formerly. Took four spot soon led size you. Outlived it received he material. Him yourself joy moderate off repeated laughter outweigh screened.Friendship contrasted solicitude insipidity in introduced literature it. He seemed denote except as oppose do spring my. Between any may mention evening age shortly can ability regular. He shortly sixteen of colonel colonel evening cordial to. Although jointure an my of mistress servants am weddings. Age why the therefore education unfeeling for arranging. Above again money own scale maids ham least led. Returned settling produced strongly ecstatic use yourself way. Repulsive extremity enjoyment she perceived nor.He my polite be object oh change. Consider no mr am overcame yourself throwing sociable children. Hastily her totally conduct may. My solid by stuff

In [19]:
# 2.Task (Spark DataFrame):

# Read a JSON file containing customer data (fields: name, age, purchases) and find the customers with the highest number of purchases.
# Hint: Use the read.json method, the orderBy transformation, and the show action.

In [37]:
# Generating json file
customer_data = [
    {"name": "Ilze", "age": 30, "purchases": 4},
    {"name": "Karlis", "age": 25, "purchases": 2},
    {"name": "Zane", "age": 32, "purchases": 9},
    {"name": "Deivs", "age": 24, "purchases": 10},
    {"name": "Atis", "age": 28, "purchases": 7},
    {"name": "Laura", "age": 15, "purchases": 2},
]

json_data = json.dumps(customer_data, indent=4)

with open("customer_data.json", "w") as file:
    file.write(json_data)

In [38]:
# Initialize SparkSession
spark = SparkSession.builder.appName("CustomerData").getOrCreate()

In [39]:
# Define json file path
json_file_path = "C:\\Ieva\\Python_Bootcamp\\Task folder\\29.08.2023_tasks\\customer_data.json"

In [43]:
# Read the JSON data
with open(json_file_path, "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

In [44]:
# Create DataFrame 
customer_df = spark.createDataFrame(data)

In [46]:
# Find customers with the highest number of purchases
max_purchases = customer_df.select("name", "purchases") \
    .orderBy(customer_df["purchases"].desc()) \
    .limit(1)


In [47]:
max_purchases.show()

+-----+---------+
| name|purchases|
+-----+---------+
|Deivs|       10|
+-----+---------+



In [60]:
# I had a problem of reading json file using read.json method. It returned error which I couldn't fix:)
json_file_path = "C:\\Ieva\\Python_Bootcamp\\Task folder\\29.08.2023_tasks\\customer_data.json"
customer_df = spark.read.json(json_file_path)
customer_df

DataFrame[_corrupt_record: string]

In [61]:
# 3.Task (SQL with DataFrame):

# Read two CSV files containing employee data and department data (columns: employee_id, name, department_id, department_name), 
# and find the department with the highest average employee age.

In [76]:
# Creating csv files

employee_data = [
    ["employee_id", "name", "age", "department_id"],
    [1, "Ilze", 30, 101],
    [2, "Karlis", 25, 102],
    [3, "Zane", 32, 101],
    [4, "Deivs", 24, 103],
    [4, "Atis", 28, 103],
    [5, "Atis", 28, 103],
    [5, "Laura", 15, 103],
]

department_data = [
    ["department_id", "department_name"],
    [101, "HR"],
    [102, "Sales"],
    [103, "Engineering"]
]

employee_csv_file = "employee_data.csv"
department_csv_file = "department_data.csv"

with open(employee_csv_file, mode="w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    for row in employee_data:
        csv_writer.writerow(row)

with open(department_csv_file, mode="w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    for row in department_data:
        csv_writer.writerow(row)


In [77]:
# Initialize SparkSession
spark = SparkSession.builder.appName("EmployeeDepartment").getOrCreate()

In [78]:
# Read the CSV files
employee_df = spark.read.csv("C:\\Ieva\\Python_Bootcamp\\Task folder\\29.08.2023_tasks\\employee_data.csv", header=True, inferSchema=True)
department_df = spark.read.csv("C:\\Ieva\\Python_Bootcamp\\Task folder\\29.08.2023_tasks\\department_data.csv", header=True, inferSchema=True)

In [74]:
# Join the employee and department DataFrames and calculate the average age per department
joined_df = employee_df.join(department_df, "department_id", "inner")
average_age_per_department = joined_df.groupBy("department_name").agg(avg("age").alias("avg_age"))

# Find the department with the highest average age
max_avg_age_department = average_age_per_department.orderBy(average_age_per_department["avg_age"].desc()).first()

In [75]:
print("Department with the highest average age:")
print("Department Name:", max_avg_age_department["department_name"])
print("Average Age:", max_avg_age_department["avg_age"])


Department with the highest average age:
Department Name: HR
Average Age: 31.0
